Mini Project - Scorberr - Scott Berry

Import dependencies

In [1]:
import numpy as np
import openpyxl
import pandas as pd
from matplotlib import pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

Add Excel Workbook to Pandas DF function

In [2]:
def workbook2df(path, first_row_header, first_column_index):
    workbook = openpyxl.load_workbook(path).active
    rows = list(workbook.iter_rows(values_only=True))
    df = pd.DataFrame(rows)
    if first_row_header:
        new_header = df.iloc[0]
        df = df[1:]
        df.columns = new_header
    if first_column_index:
        df = df.set_index("SL")
    return df

Import Data

In [3]:
games_to_win = workbook2df("data/gamesToWin.xlsx", True, True)
sl_matchup_data = workbook2df("data/slmatchups/SLMatchups.xlsx", True, False)
# player_data = workbook2df("data/wookieMistakesPlayerData.xlsx", True, False)
# game_data = workbook2df("data/wookieMistakesSpring2022Games.xlsx", True, False)

FileNotFoundError: [Errno 2] No such file or directory: 'data/gamesToWin.xlsx'

Get Expected Points in every Skill Level matchup

In [ ]:
def get_sl_matchup_stats(df, games2win):
    slmatches = games2win
    slrange = range(2, 8)
    for p1skill in slrange:
        for p2skill in slrange:
            matchup_data = SLMatches(p1skill, p2skill)
            for index, row in df.iterrows():
                if row['Player_1'] == p1skill and row['Player_2'] == p2skill:
                    matchup_data.addpoints(row['Points_1'])
                    matchup_data.addgame(index)
                    if p1skill == p2skill:
                        matchup_data.addpoints(row['Points_2'])
            for index, row in df.iterrows():
                if index not in matchup_data.indices_checked:
                    if row['Player_2'] == p1skill and row['Player_1'] == p2skill:
                        matchup_data.addpoints(row['Points_2'])
                        matchup_data.addgame(index)
            matchup_data.getaverage()
            if p1skill == p2skill:
                matchup_data.average /= 2
            if matchup_data.games < 6:
                matchup_data.average = "X{}".format(round(matchup_data.average, 2))
            try:
                slmatches.loc[p1skill, p2skill] = round(matchup_data.average, 2)
            except TypeError:
                slmatches.loc[p1skill, p2skill] = matchup_data.average
    for sl in slrange:
        plt.plot(np.array(slrange), np.array([float(str(x).replace("X", "", 1)) for x in slmatches.values[sl-2]]))
        title = 'Matchups SL {}'.format(sl)
        imgtitle = "data/slmatchups/slmatchups{}.jpg".format(sl)
        plt.title(title)
        plt.xlabel('Opponent SL')
        plt.ylabel('Average Points')
        plt.savefig(imgtitle)
        plt.show()
    slmatches.to_excel(r'data/slmatchups/SLMatchupAverages.xlsx', index=True, header=True)
    print(slmatches)


get_sl_matchup_stats(sl_matchup_data, games_to_win)